# Test full calculation
Test the full calculation of the 

In [2]:
pip install xmip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 1.9 MB/s eta 0:00:00
  Attempting uninstall: xgcm
    Found existing installation: xgcm 0.8.0
    Uninstalling xgcm-0.8.0:
      Successfully uninstalled xgcm-0.8.0
Note: you may need to restart the kernel to use updated packages.


## Import packages

In [3]:
%matplotlib inline
import xarray as xr
from xmip.preprocessing import rename_cmip6
import s3fs
import zarr

import util

## Summary of some test data

| model name | resolution | years in file for one variable | file size (one variable) | combined 3D variable data size | combined 2D variable data size |
| --- | --- | --- | --- | --- | --- |
| ACCESS-OM2 | 100 km | 10 | 670 MB | 17 GB | 0.4 GB |
| ACCESS-OM2-025 | 25 km | 1 | 1.1 GB | 271 GB | 7 GB |
| CMCC-CM2-SR5 | 100 km | 61 | 3.6 GB | 15 GB | 0.4 GB |
| CMCC-CM2-HR4 | 25 km | 10 | 9.4 GB | 231 GB | 6 GB |

## Load data from ESGF

In [4]:
result = util.esgf_search(activity_id='OMIP', experiment_id='omip2', variable_id='vmo', source_id='ACCESS-OM2')#CMCC-CM2-SR5')

https://esgf-node.llnl.gov/esg-search/search/?activity_id=OMIP&experiment_id=omip2&variable_id=vmo&source_id=ACCESS-OM2&project=CMIP6&type=File&distrib=false&format=application%2Fsolr%2Bjson&offset=0
https://esgf-node.llnl.gov/esg-search/search/?activity_id=OMIP&experiment_id=omip2&variable_id=vmo&source_id=ACCESS-OM2&project=CMIP6&type=File&distrib=false&format=application%2Fsolr%2Bjson&offset=10
https://esgf-node.llnl.gov/esg-search/search/?activity_id=OMIP&experiment_id=omip2&variable_id=vmo&source_id=ACCESS-OM2&project=CMIP6&type=File&distrib=false&format=application%2Fsolr%2Bjson&offset=20
https://esgf-node.llnl.gov/esg-search/search/?activity_id=OMIP&experiment_id=omip2&variable_id=vmo&source_id=ACCESS-OM2&project=CMIP6&type=File&distrib=false&format=application%2Fsolr%2Bjson&offset=30
https://esgf-node.llnl.gov/esg-search/search/?activity_id=OMIP&experiment_id=omip2&variable_id=vmo&source_id=ACCESS-OM2&project=CMIP6&type=File&distrib=false&format=application%2Fsolr%2Bjson&offset

In [5]:
# pick one file for testing
files_to_open = [result[-1]]

In [6]:
ds = xr.open_mfdataset(files_to_open, combine='by_coords')

/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/coding/times.py:682: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/core/indexing.py:524: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return np.asarray(array[self.key], dtype=None)


## Pre-proc with `XMIP`

In [7]:
ds = rename_cmip6(ds)

In [8]:
ds

<xarray.Dataset>
Dimensions:      (time: 12, bnds: 2, lev: 50, y: 300, x: 360, vertex: 4)
Coordinates:
  * time         (time) object 0366-01-16 12:00:00 ... 0366-12-16 12:00:00
  * lev          (lev) float64 1.152 3.649 6.565 ... 5.034e+03 5.254e+03
  * y            (y) int32 0 1 2 3 4 5 6 7 8 ... 292 293 294 295 296 297 298 299
  * x            (x) int32 0 1 2 3 4 5 6 7 8 ... 352 353 354 355 356 357 358 359
Dimensions without coordinates: bnds, vertex
Data variables:
    time_bounds  (time, bnds) object dask.array<chunksize=(12, 2), meta=np.ndarray>
    lev_bounds   (lev, bnds) float64 dask.array<chunksize=(50, 2), meta=np.ndarray>
    lat_bounds   (y, x, vertex) float64 dask.array<chunksize=(300, 360, 4), meta=np.ndarray>
    lon_bounds   (y, x, vertex) float64 dask.array<chunksize=(300, 360, 4), meta=np.ndarray>
    lat          (y, x) float64 dask.array<chunksize=(300, 360), meta=np.ndarray>
    lon          (y, x) float64 dask.array<chunksize=(300, 360), meta=np.ndarray>
    vmo          (time, lev, y, x) float32 dask.array<chunksize=(12, 50, 300, 360), meta=np.ndarray>
Attributes: (12/47)
    Conventions:                     CF-1.7 CMIP-6.2
    activity_id:                     OMIP
    branch_method:                   no parent
    branch_time_in_child:            0.0
    branch_time_in_parent:           0.0
    creation_date:                   2021-06-16T00:40:35Z
    ...                              ...
    variable_id:                     vmo
    variant_label:                   r1i1p1f1
    version:                         v20210616
    license:                         CMIP6 model data produced by CSIRO is li...
    cmor_version:                    3.4.0
    DODS_EXTRA.Unlimited_Dimension:  time

In [9]:
def add_geo(DS):
    # assign geolat and geolon from ocean static
    DS = DS.assign_coords({'lon':DS['lon'].sel(x=DS.x,y=DS.y),
                           'lat':DS['lat'].sel(x=DS.x,y=DS.y)})
    return DS

In [10]:
ds = add_geo(ds)

In [11]:
ds

<xarray.Dataset>
Dimensions:      (time: 12, bnds: 2, lev: 50, y: 300, x: 360, vertex: 4)
Coordinates:
  * time         (time) object 0366-01-16 12:00:00 ... 0366-12-16 12:00:00
  * lev          (lev) float64 1.152 3.649 6.565 ... 5.034e+03 5.254e+03
  * y            (y) int32 0 1 2 3 4 5 6 7 8 ... 292 293 294 295 296 297 298 299
  * x            (x) int32 0 1 2 3 4 5 6 7 8 ... 352 353 354 355 356 357 358 359
    lat          (y, x) float64 dask.array<chunksize=(300, 360), meta=np.ndarray>
    lon          (y, x) float64 dask.array<chunksize=(300, 360), meta=np.ndarray>
Dimensions without coordinates: bnds, vertex
Data variables:
    time_bounds  (time, bnds) object dask.array<chunksize=(12, 2), meta=np.ndarray>
    lev_bounds   (lev, bnds) float64 dask.array<chunksize=(50, 2), meta=np.ndarray>
    lat_bounds   (y, x, vertex) float64 dask.array<chunksize=(300, 360, 4), meta=np.ndarray>
    lon_bounds   (y, x, vertex) float64 dask.array<chunksize=(300, 360, 4), meta=np.ndarray>
    vmo          (time, lev, y, x) float32 dask.array<chunksize=(12, 50, 300, 360), meta=np.ndarray>
Attributes: (12/47)
    Conventions:                     CF-1.7 CMIP-6.2
    activity_id:                     OMIP
    branch_method:                   no parent
    branch_time_in_child:            0.0
    branch_time_in_parent:           0.0
    creation_date:                   2021-06-16T00:40:35Z
    ...                              ...
    variable_id:                     vmo
    variant_label:                   r1i1p1f1
    version:                         v20210616
    license:                         CMIP6 model data produced by CSIRO is li...
    cmor_version:                    3.4.0
    DODS_EXTRA.Unlimited_Dimension:  time

## Subset Pan-Arctic (lat>50N)

In [12]:
cond = (ds['lat']<=50)
dsnow = ds[['vmo']].where(cond,drop=True)

In [13]:
dsnow = dsnow.chunk(chunks={'x':25,'y':25})

In [14]:
dsnow

<xarray.Dataset>
Dimensions:  (time: 12, lev: 50, y: 219, x: 360)
Coordinates:
  * time     (time) object 0366-01-16 12:00:00 ... 0366-12-16 12:00:00
  * lev      (lev) float64 1.152 3.649 6.565 ... 4.815e+03 5.034e+03 5.254e+03
  * y        (y) int32 0 1 2 3 4 5 6 7 8 ... 210 211 212 213 214 215 216 217 218
  * x        (x) int32 0 1 2 3 4 5 6 7 8 ... 351 352 353 354 355 356 357 358 359
    lat      (y, x) float64 dask.array<chunksize=(25, 25), meta=np.ndarray>
    lon      (y, x) float64 dask.array<chunksize=(25, 25), meta=np.ndarray>
Data variables:
    vmo      (time, lev, y, x) float32 dask.array<chunksize=(12, 50, 25, 25), meta=np.ndarray>
Attributes: (12/47)
    Conventions:                     CF-1.7 CMIP-6.2
    activity_id:                     OMIP
    branch_method:                   no parent
    branch_time_in_child:            0.0
    branch_time_in_parent:           0.0
    creation_date:                   2021-06-16T00:40:35Z
    ...                              ...
    variable_id:                     vmo
    variant_label:                   r1i1p1f1
    version:                         v20210616
    license:                         CMIP6 model data produced by CSIRO is li...
    cmor_version:                    3.4.0
    DODS_EXTRA.Unlimited_Dimension:  time

In [15]:
from dask_gateway import Gateway
gateway = Gateway()

In [16]:
##A line of trick to clean your dask cluster before you start your computation
from dask.distributed import Client
clusters=gateway.list_clusters()
print(clusters )
for cluster in clusters :
    cluster= gateway.connect(cluster.name)
    print(cluster)
    client = Client(cluster)
    client.close()
    cluster.shutdown()

[]


### Create a new Dask cluster with the Dask Gateway

In [17]:
cluster = gateway.new_cluster(worker_memory=2, worker_cores=1)

cluster.scale(5)
cluster

### Get a client from the Dask Gateway Cluster

As stated above, creating a Dask `Client` is mandatory in order to perform following Daks computations on your Dask Cluster.

In [18]:
from distributed import Client

if cluster:
    client = Client(cluster) # create a dask Gateway cluster
else:
    client = Client()   # create a local dask cluster on the machine.
client

Connection method: Cluster object,Cluster type: dask_gateway.GatewayCluster
Dashboard: /jupyterhub/services/dask-gateway/clusters/daskhub.7bece9355c8a412aac9561c17720c485/status,


In [19]:
your_name='tinaok'
path='WAFFLES/'+your_name
s3_prefix =  "s3://"+path
print(s3_prefix)
access_key = !aws configure get aws_access_key_id
access_key = access_key[0]
secret_key = !aws configure get aws_secret_access_key
secret_key = secret_key[0]
client_kwargs={'endpoint_url': 'https://object-store.cloud.muni.cz'}

s3://WAFFLES/tinaok


In [20]:
zarr_file_name= "FSStore_zarr"
uri = f"{s3_prefix}/{zarr_file_name}"
store = zarr.storage.FSStore(uri, client_kwargs=client_kwargs,key=access_key, secret=secret_key)
store.map.fs.find(uri)

[]

In [21]:
%time dsnow.to_zarr(store=store, mode='w', consolidated=True)

CPU times: user 1.93 s, sys: 209 ms, total: 2.14 s
Wall time: 3min 54s


In [24]:
target = s3fs.S3FileSystem(anon=False,
      client_kwargs={
         'endpoint_url': 'https://object-store.cloud.muni.cz'
      })

In [25]:
remote_path =path
target.ls(remote_path)

['WAFFLES/tinaok/FSStore_zarr']

In [26]:
xr.open_zarr(store=store)#consolidated=True)

/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/coding/times.py:682: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/core/indexing.py:524: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return np.asarray(array[self.key], dtype=None)


<xarray.Dataset>
Dimensions:  (y: 219, x: 360, lev: 50, time: 12)
Coordinates:
    lat      (y, x) float64 dask.array<chunksize=(25, 25), meta=np.ndarray>
  * lev      (lev) float64 1.152 3.649 6.565 ... 4.815e+03 5.034e+03 5.254e+03
    lon      (y, x) float64 dask.array<chunksize=(25, 25), meta=np.ndarray>
  * time     (time) object 0366-01-16 12:00:00 ... 0366-12-16 12:00:00
  * x        (x) int32 0 1 2 3 4 5 6 7 8 ... 351 352 353 354 355 356 357 358 359
  * y        (y) int32 0 1 2 3 4 5 6 7 8 ... 210 211 212 213 214 215 216 217 218
Data variables:
    vmo      (time, lev, y, x) float32 dask.array<chunksize=(12, 50, 25, 25), meta=np.ndarray>
Attributes: (12/47)
    Conventions:                     CF-1.7 CMIP-6.2
    DODS_EXTRA.Unlimited_Dimension:  time
    activity_id:                     OMIP
    branch_method:                   no parent
    branch_time_in_child:            0.0
    branch_time_in_parent:           0.0
    ...                              ...
    table_info:                      Creation Date:(30 April 2019) MD5:12ec21...
    title:                           ACCESS-OM2 output prepared for CMIP6
    tracking_id:                     hdl:21.14100/1376e4bb-ca93-45b5-9d98-8cf...
    variable_id:                     vmo
    variant_label:                   r1i1p1f1
    version:                         v20210616

## Extract Berring Strait

## Calc Volume Transport